In [1]:
import pandas as pd
import numpy as np
import datetime

from functools import reduce

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('output/df_house_info_2.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
loc_cols = [
    'locator_class',
    'locator_importance',
    'lat',
    'lon',
    'locator_osm_type',
    'locator_type',
    'post_index',
    'Kaliningrad',
    'Zelenogradsk',
    'Svetlogorsk',
    'Gurevsk',
    'Baltiisk',
    'Kaliningrad_center',
    'school_0_id',
    'school_0_forma',
    'school_0_type',
    'school_0_dist',
    'school_1_id',
    'school_1_forma',
    'school_1_type',
    'school_1_dist',
    'school_2_id',
    'school_2_forma',
    'school_2_type',
    'school_2_dist',
    'parks_maks_ashman',
    'parks_botan_garden',
    'parks_yunost',
    'parks_old_park',
    'parks_upper_lake',
    'parks_lower_lake',
    'parks_kant_island',
    'parks_south_park',
    'parks_central_park',
    'parks_pobeda_park',
    'parks_ocean_museum_park',
    'stations_severny_station',
    'stations_south_station',
    'airports_chkalovsk_ari',
    'airports_chrabrovo',
    'airports_devau',
    'airports_gvardeisk',
    'airports_severny',
    'airports_medovoe',
    'airports_strelnya',
    'airports_valdau',
    'airports_dunaevka',
    'airports_svetlogorsk',
]

old_first_info_cols = [
    'mun_obr_oktmo',
    'mun_obr',
    'mkd_code',
    'houseguid',
    'commission_year',
    'architectural_monument_status',
    'architectural_monument_category_id',
    'total_sq',
    'overhaul_fee_sq',
    'total_rooms_amount',
    'living_rooms_amount',
    'living_rooms_with_nonresidental_amount',
    'total_rooms_sq',
    'living_rooms_sq',
    'living_rooms_with_nonresidental_sq',
    'total_ppl',
    'number_floors_max',
    'number_floors_min',
    'money_collecting_way',
    'money_collecting_way_date_decision',
    'bank_bik',
    'money_ppl_collected',
    'loan_balance',
    'money_ppl_collected_debts',
    'overhaul_funds_spent_all',
    'overhaul_funds_spent_subsidy',
    'overhaul_fund_spent_other',
    'overhaul_funds_balance',
    'update_date_of_information',
    'money_ppl_collected_date',
    'owners_payment',
    'energy_efficiency',
    'previous_energy_efficiency',
    'energy_audit_date',
    'is_change_energy_efficiency',
    'alarm_document_date',
    'exclude_date_from_program',
    'reason_of_exclude_from_program_id',
    'inclusion_date_to_program',
    'comment',
    'last_update',
    'house_id',
]

first_info_cols = [f'first_{col}' for col in old_first_info_cols]

second_info_cols = [
    'second_info_management_organization_id',
    'second_info_built_year',
    'second_info_exploitation_start_year',
    'second_info_project_type',
    'second_info_house_type',
    'second_info_is_alarm',
    'second_info_method_of_forming_overhaul_fund',
    'second_info_floor_count_max',
    'second_info_floor_count_min',
    'second_info_entrance_count',
    'second_info_elevators_count',
    'second_info_energy_efficiency',
    'second_info_quarters_count',
    'second_info_living_quarters_count',
    'second_info_unliving_quarters_count',
    'second_info_area_total',
    'second_info_area_residential',
    'second_info_area_non_residential',
    'second_info_area_common_property',
    'second_info_area_land',
    'second_info_parking_square',
    'second_info_playground',
    'second_info_sportsground',
    'second_info_other_beautification',
    'second_info_foundation_type',
    'second_info_floor_type',
    'second_info_wall_material',
    'second_info_basement_area',
    'second_info_chute_type',
    'second_info_chute_count',
    'second_info_electrical_type',
    'second_info_electrical_entries_count',
    'second_info_heating_type',
    'second_info_hot_water_type',
    'second_info_cold_water_type',
    'second_info_sewerage_type',
    'second_info_sewerage_cesspools_volume',
    'second_info_gas_type',
    'second_info_ventilation_type',
    'second_info_firefighting_type',
    'second_info_drainage_type',
]

In [5]:
df.rename(columns=dict(zip(old_first_info_cols, first_info_cols)), inplace=True)

# 1. Parsed cols

In [6]:
clean_df = df[[
    'new_object', 
    'dt',
    'n_rooms',
    'type',
    'price',
    'area',
    'price_per_metr',
    'floor',
    'max_floor',
    'is_available',
    'available_date'
]]

# 2. Address Parsing

#### Our addres components were used a lot in previous notebooks, so it does not worse to parse address again

In [21]:
from preprocessing import get_address_components

address = df[['address']]
address.columns = ['labels']

address_components = address.apply(get_address_components, axis=1)
address_components.drop('region', axis=1, inplace=True)

clean_df = clean_df.join(address_components)

# 3. Location

In [22]:
clean_df = clean_df.join(df[loc_cols])

# 4. First info

In [23]:
first_info_cat_cols = [
    'first_architectural_monument_status',
    'first_money_collecting_way',
    'first_comment',
]

first_info_num_cols = [
    "first_commission_year",
    'first_total_sq',
    'first_overhaul_fee_sq',
    'first_total_rooms_amount',
    'first_living_rooms_amount',
    'first_living_rooms_with_nonresidental_amount',
    'first_total_rooms_sq',
    'first_living_rooms_sq',
    'first_living_rooms_with_nonresidental_sq',
    'first_total_ppl',
    'first_money_ppl_collected',
    'first_money_ppl_collected_debts',
    'first_overhaul_funds_spent_all',
    'first_overhaul_funds_spent_subsidy',
    'first_overhaul_funds_balance',
]

first_info_date_cols = [
    "first_money_collecting_way_date_decision",
    'first_money_ppl_collected_date',
    'first_alarm_document_date',
    'first_exclude_date_from_program',
]



first_info_cols = [
    *first_info_cat_cols,
    *first_info_num_cols,
    *first_info_date_cols,
]

In [50]:
def str2tuple(val):
    if pd.isna(val):
        return val,
    tup = tuple(map(lambda x: np.nan if (pd.isna(x) or x=='nan') else x, val.strip('[]').split(', ')))
    if len(set(tup)) == 1:
        tup = (tup[0],)
    return tup

def catlabelencoder(series):
    series = series.map(
        lambda x: tuple(pd.Series(x).map(lambda x: x.lower(), 'ignore').mode())
        if len(x) != 1 else x
    )
    
    label_dict = {(np.nan,):np.nan}
    uniq_vals = [val for val in series.unique() if not (val in label_dict)]
    cat_vals = [f'cat_{i}' for i in range(len(uniq_vals))]
    label_dict.update(dict(zip(uniq_vals, cat_vals)))
    new_series = series.map(label_dict.get)
    return new_series

def get_num_val(vals):
    if len(vals) == 1:
        val = vals[0]
        if pd.isna(val):
            return val
        else:
            return float(val.replace(',', '.').replace("'", ""))
    med = np.median(list(map(lambda x: float(
        x.replace(',', '.')
        .replace("'", "")
    ), filter(lambda x: not pd.isna(x), vals))))
    
    return med

def prep_date(str_date):
    str_date = str_date.replace("'", "").replace('.', '-')
    if len(str_date.split('-')[0]) != 4:
        str_date = '-'.join(str_date.split('-')[::-1])
    return datetime.date.fromisoformat(str_date)
    
def get_date_val(vals):
    if len(vals) == 1:
        val = vals[0]
        if pd.isna(val):
            return val
        else:
            return prep_date(val)
    vals = list(map(prep_date, filter(lambda x: not pd.isna(x), vals)))
    
    date = vals[0]
    diff = [(date - val).days for val in vals]
    
    med = int(np.median(diff))
    
    return date + datetime.timedelta(med)

In [25]:
first_info = df[first_info_cols]
first_info = first_info.applymap(str2tuple)

first_info[first_info_cat_cols] = first_info[first_info_cat_cols].apply(catlabelencoder)
first_info[first_info_num_cols] =  first_info[first_info_num_cols].applymap(get_num_val)
first_info[first_info_date_cols] = first_info[first_info_date_cols].applymap(lambda x: pd.to_datetime(get_date_val(x)))

In [26]:
first_rename_cols_dict = {
    **{i: f"first_cat_{i[6:]}" for i in first_info_cat_cols},
    **{i: f"first_num_{i[6:]}" for i in first_info_num_cols},
    **{i: f"first_dt_{i[6:]}" for i in first_info_date_cols},
}
first_info.rename(columns=first_rename_cols_dict, inplace=True)

# 5. Second info columns

In [84]:
second_info_cat_cols = [
    'second_info_project_type',
    'second_info_house_type',
    'second_info_is_alarm',
    'second_info_method_of_forming_overhaul_fund',
    'second_info_energy_efficiency',
    'second_info_other_beautification',
    'second_info_foundation_type',
    'second_info_floor_type',
    'second_info_wall_material',
    'second_info_chute_type',
    'second_info_electrical_type',
    'second_info_heating_type',
    'second_info_hot_water_type',
    'second_info_cold_water_type',
    'second_info_sewerage_type',
    'second_info_gas_type',
    'second_info_ventilation_type',
    'second_info_firefighting_type',
    'second_info_drainage_type',
]

second_info_num_cols = [
    'second_info_built_year',
    'second_info_exploitation_start_year',
    'second_info_entrance_count',
    'second_info_elevators_count',
    'second_info_quarters_count',
    'second_info_living_quarters_count',
    'second_info_unliving_quarters_count',
    'second_info_area_total',
    'second_info_area_residential',
    'second_info_area_non_residential',
    'second_info_area_common_property',
    'second_info_area_land',
    'second_info_parking_square',
    'second_info_playground',
    'second_info_sportsground',
    'second_info_basement_area',
    'second_info_chute_count',
    'second_info_electrical_entries_count',
    'second_info_sewerage_cesspools_volume',
]

second_info_cols = [
    *second_info_cat_cols,
    *second_info_num_cols,
]

In [85]:
second_info = df[second_info_cols]
second_info = second_info.applymap(str2tuple)

second_info[second_info_cat_cols] = second_info[second_info_cat_cols].apply(catlabelencoder)
second_info[second_info_num_cols] =  second_info[second_info_num_cols].applymap(get_num_val)

In [86]:
second_rename_cols_dict = {
    **{i: f"second_cat_{i[12:]}" for i in second_info_cat_cols},
    **{i: f"second_num_{i[12:]}" for i in second_info_num_cols},
}
second_info.rename(columns=second_rename_cols_dict, inplace=True)

# 6. Join and Save

In [90]:
clean_df = reduce(lambda x, y: x.join(y), [clean_df, first_info, second_info])

In [91]:
clean_df.to_csv('output/final_dataset.csv', index=False)